In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard
import tensorflow as tf
import numpy as np
from  datetime import datetime

2025-01-14 13:58:17.525695: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-14 13:58:17.678431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736859497.733552    8114 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736859497.751130    8114 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-14 13:58:17.898836: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
model = tf.saved_model.load("../models/camera0/segmentation/saved_model")

W0000 00:00:1736859506.683081    8114 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
sig0 = model.signatures["serving_default"].structured_input_signature
display(sig0)
sig = (
    (),
    {
        "input_image": sig0[1]["input_image"],
        "input_image_meta": sig0[1]["input_image_meta"],
        "input_anchors": sig0[1]["input_anchors"],
    },
)
sig

((),
 {'input_anchors': TensorSpec(shape=(None, None, 4), dtype=tf.float32, name='input_anchors'),
  'input_image': TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='input_image'),
  'input_image_meta': TensorSpec(shape=(None, 14), dtype=tf.float32, name='input_image_meta')})

((),
 {'input_image': TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='input_image'),
  'input_image_meta': TensorSpec(shape=(None, 14), dtype=tf.float32, name='input_image_meta'),
  'input_anchors': TensorSpec(shape=(None, None, 4), dtype=tf.float32, name='input_anchors')})

In [26]:
from pathlib import Path

with Path("./input.py.bin").open() as f:
    c = f.read()
exec(c)
images: np.ndarray
image_metas: np.ndarray
anchors: np.ndarray

images = images.astype(np.float32)
image_metas = image_metas.astype(np.float32)
anchors = anchors.astype(np.float32)
display(anchors)
print("image_metas", image_metas)
display(detections.shape)
display(detections)

array([[[-0.02211869, -0.01105934,  0.02114117,  0.01008183],
        [-0.01564027, -0.01564027,  0.01466276,  0.01466276],
        [-0.01105934, -0.02211869,  0.01008183,  0.02114117],
        ...,
        [ 0.5845174 ,  0.7614669 ,  1.2913378 ,  1.1143883 ],
        [ 0.68817204,  0.68817204,  1.1876833 ,  1.1876833 ],
        [ 0.7614669 ,  0.5845174 ,  1.1143883 ,  1.2913378 ]]],
      dtype=float32)

image_metas [[0.000e+00 1.024e+03 1.024e+03 3.000e+00 1.024e+03 1.024e+03 3.000e+00
  0.000e+00 0.000e+00 1.024e+03 1.024e+03 1.000e+00 0.000e+00 0.000e+00]]


(1, 10, 6)

array([[[0.31554943, 0.60850537, 0.45372224, 0.69937855, 1.        ,
         0.99838877],
        [0.37167203, 0.4351973 , 0.39085433, 0.4648971 , 1.        ,
         0.8981155 ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ]]], dtype=float32)

In [27]:
display(anchors.shape)
# anchors = anchors.reshape(-1)
# anchors[:] = range(anchors.shape[0])
# anchors=anchors.reshape((1,261888,4))
display(anchors[0,0:4,0:4])
with Path("../models/camera0/segmentation/anchors.bin").open("bw") as f:
    f.write(anchors.tobytes())

(1, 261888, 4)

array([[-0.02211869, -0.01105934,  0.02114117,  0.01008183],
       [-0.01564027, -0.01564027,  0.01466276,  0.01466276],
       [-0.01105934, -0.02211869,  0.01008183,  0.02114117],
       [-0.02211869, -0.00714928,  0.02114117,  0.0139919 ]],
      dtype=float32)

In [28]:
input_image = tf.convert_to_tensor(images.astype(np.float32), name="input_image")
# input_image = tf.convert_to_tensor(np.random.uniform(0,1,size=(1,1024,1024,3)).astype(np.float32), name="input_image")

input_image_meta = tf.convert_to_tensor(
    image_metas.astype(np.float32), name="input_image_meta"
)
input_anchors = tf.convert_to_tensor(anchors.astype(np.float32), name="input_anchors")

result = model([input_image, input_image_meta, input_anchors])
display(result[0].shape)
display(result[1].shape)
display(result)

W0000 00:00:1736866716.873729    8114 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "CropAndResize" attr { key: "T" value { type: DT_FLOAT } } attr { key: "extrapolation_value" value { f: 0 } } attr { key: "method" value { s: "bilinear" } } inputs { dtype: DT_FLOAT shape { dim { size: 1 } dim { size: 256 } dim { size: 256 } dim { size: 256 } } } inputs { dtype: DT_FLOAT shape { dim { size: -20 } dim { size: 4 } } } inputs { dtype: DT_INT32 shape { dim { size: -20 } } } inputs { dtype: DT_INT32 shape { dim { size: 2 } } value { dtype: DT_INT32 tensor_shape { dim { size: 2 } } int_val: 7 } } device { type: "CPU" vendor: "GenuineIntel" model: "103" frequency: 1996 num_cores: 32 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 49152 l2_cache_size: 2097152 l3_cache_size: 37748736 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { dim { size: -20 } dim { 

TensorShape([1, 10, 6])

TensorShape([1, 1000, 2])

[<tf.Tensor: shape=(1, 10, 6), dtype=float32, numpy=
 array([[[0.3155494 , 0.60850537, 0.4537222 , 0.69937855, 1.        ,
          0.99838877],
         [0.37167203, 0.4351973 , 0.39085433, 0.4648971 , 1.        ,
          0.8981158 ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ],
         [0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        ]]], dtype=float32)>

In [32]:
import onnx

onnx_model_path = "../models/camera0/segmentation/saved_model.onnx"
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model,full_check=True)

import onnxruntime as ort

ort_sess = ort.InferenceSession(onnx_model_path, providers=["CUDAExecutionProvider"])
outputs = ort_sess.run(
    None,
    {"input_image": images, "input_image_meta": image_metas, "input_anchors": anchors},
)

# Print Result
for i, output in enumerate(outputs):
    print(f"Output #{i}")
    display(type(output))
    display(output.shape)
    display(output[0, 0])

2025-01-14 16:24:30.378278069 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 29 Memcpy nodes are added to the graph tf2onnx for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-01-14 16:24:30.378469423 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 5 Memcpy nodes are added to the graph tf2onnx__2553 for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-01-14 16:24:30.378603169 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 5 Memcpy nodes are added to the graph tf2onnx__2621 for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-01-14 16:2

Output #0


numpy.ndarray

(1, 1000, 4)

array([0.3164205 , 0.6093571 , 0.45560375, 0.70326054], dtype=float32)

...

array([ 0.11700051, -0.28296793, -1.7756131 , -1.5606557 ], dtype=float32)

Output #6


numpy.ndarray

(1, 261888, 2)

array([0.977842  , 0.02215798], dtype=float32)